1) test1.find_opt() - Обычный поиск
2) test1.find_opt_change() - Поиск с изменением границ компонентов и соотношений.
3) test1.find_opt_change_comp_bond() - Поиск с изменением границ компонетов
4) test1.find_opt_change_ratio() - Поиск с изменением соотношений
5) test1.find_opt_change_one() - Поиск с изменением границ компонентов и соотношений. Перебираются последовательно. Даётся несколько выриантов

In [1]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np

In [2]:
df=pd.read_excel('data1.xlsx')
df

,Unnamed: 0,Имя,Цена,Сухое,Натура,Сырой,Крахмал,NDF,ADF,Ca,Phos
0,Имся,единица,руб,кг,кг,кг,кг,кг,кг,кг,кг
1,Ячмень зерно,кг,10.000,0.900,1.111,0.150,0.500,0.200,0.100,0.002,0.003
2,Кукуруза зерно,кг,15.000,0.800,1.250,0.100,0.600,0.250,0.150,0.003,0.004
3,Силос,кг,5.000,0.400,2.500,0.050,0.300,0.400,0.200,0.004,0.005
4,Соя шрот,кг,20.000,0.900,1.111,0.400,0.200,0.300,0.250,0.005,0.006


In [3]:
class method1():   
    def __init__(self, products, bnd, comps, comps_bnd, matrix, costs, sootnosheniya=None):
        self.products=products
        self.bnd=bnd
        self.comps=comps
        self.comps_bnd=comps_bnd
        self.ratio=sootnosheniya
        self.main_mat=matrix[products, :][:, comps]
        cond_l=[]
        cond_r=[]
        if comps_bnd:
            for num, i in enumerate(comps_bnd):
                if i[0]:
                    cond_l.append(-self.main_mat[:, num])
                    cond_r.append(-i[0])
                if i[1]:
                    cond_l.append(self.main_mat[:, num])
                    cond_r.append(i[1])
        self.cond_l=np.array(cond_l)
        self.cond_r=np.array(cond_r)
        self.costs=costs
        self.result=None
        self.KOEF=1.03

    def get_ratio(products, comps, sootnosheniya, main_mat):
        for_ratio=np.zeros([len(sootnosheniya), len(products)])
        for num, i in enumerate(sootnosheniya):
            chisl=0
            znam=0
            idx1=0
            idx2=0
            for num1, j in enumerate(i):
                if j!=0:
                    if j!=1 and j!=-1:
                        chisl=j
                        idx1=num1
                    else:
                        znam=j
                        idx2=num1
            for_ratio[num]=main_mat[:, idx2]*chisl-main_mat[:, idx1]*znam
        return for_ratio
        
    def find_opt(self, ratio=None, cond_r=None, mode=1):
        if ratio is None and not self.ratio is None:
            ratio=self.ratio
        if cond_r is None and len(self.cond_r)!=0:
            cond_r=self.cond_r   
        
        if not self.ratio is None and mode==1:
            if len(self.cond_l)>0:
                ratio_stats=method1.get_ratio(products, comps, ratio, self.main_mat)
                A_ub=np.concatenate([self.cond_l, ratio_stats], axis=0)
                b_ub=np.concatenate([cond_r, np.zeros(len(ratio_stats))], axis=0)
            else:
                A_ub=ratio_stats
                b_ub=self.np.zeros(len(ratio_stats))
        else:
            A_ub=self.cond_l
            b_ub=cond_r
            
        opt = linprog(c=self.costs, 
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=self.bnd,
        method="highs")
        if opt.success:
            result=np.around(np.matmul(opt.x, self.main_mat),3)
            x=np.around(opt.x, 3)
            fun=round(opt.fun, 3)           
        else:
            result=None
            x=None
            fun=None
        if not ratio is None:
            ratio=np.around(ratio, 3)       
        return {'success':opt.success, 'x': x, 'fun':fun, 'comps': result,
               'ratio': ratio, 'comps_bnd': np.around(cond_r, 3)}
        
    def change_bnd(self, cond_r):
        cond_r=np.where(cond_r[:]>0, cond_r[:]*self.KOEF, cond_r[:]/self.KOEF)
        return cond_r 
        
    def find_opt_change_comp_bond(self):
        counter=1
        cond_r=self.cond_r
        while not self.find_opt(cond_r=cond_r)['success']:
            cond_r=self.change_bnd(cond_r)
            #print('№', counter, cond_r)
            counter+=1
        return self.find_opt(cond_r=cond_r)
     
    def change_ratio(self, ratio):       
        ratio=np.where((ratio[:]>0) & (ratio[:]!=1), ratio[:]/self.KOEF, ratio[:])
        ratio=np.where(ratio[:]<0, ratio[:]*self.KOEF, ratio[:])
        return ratio
    
    def find_opt_change_ratio(self):
        counter=1
        ratio=np.array(self.ratio)
        while not self.find_opt(ratio=ratio)['success']:
            ratio=self.change_ratio(ratio)
            #print('№', counter, ratio)
            counter+=1
        return self.find_opt(ratio=ratio)
    
    def find_opt_change(self):        
        if self.find_opt(mode=2):#Если без ограничений имеется решение
            counter=1
            ratio=np.array(self.ratio)
            cond_r=self.cond_r
            while not self.find_opt(ratio=ratio, cond_r=cond_r)['success']:#В цикле сдвигаем границы
                ratio=self.change_ratio(ratio)
                cond_r=self.change_bnd(cond_r)
                #print('№', counter)
                #print(ratio, cond_r)
                counter+=1
            return self.find_opt(ratio=ratio, cond_r=cond_r)
        return  {'success':False}
    
    def find_opt_change_one(self):
        
        for i in range(len(self.ratio)):
            ratio=np.array(self.ratio)
            cond_r=self.cond_r
            dis_ratio=np.concatenate([ratio[:i,:], ratio[i+1:,:]], axis=0)
            res=self.find_opt(ratio=dis_ratio, cond_r=cond_r)
            if res['success']:
                print(f'Изменение соотношения {i+1}')
                while not self.find_opt(ratio=ratio, cond_r=cond_r)['success']:#В цикле сдвигаем границы
                    ratio[i]=np.where((ratio[i,:]>0) & (ratio[i,:]!=1), ratio[i,:]/self.KOEF, ratio[i,:])
                    ratio[i]=np.where(ratio[i,:]<0, ratio[i,:]*self.KOEF, ratio[i,:])
                print(self.find_opt(ratio=ratio, cond_r=cond_r))
        for i in range(len(self.cond_r)):
            ratio=np.array(self.ratio)
            cond_r=self.cond_r.copy()
            cond_r_max=cond_r.copy()
            cond_r_max[i]=np.where(cond_r[i]>0, cond_r[i]*100, 0)
            res=self.find_opt(ratio=ratio, cond_r=cond_r_max)
            if res['success']:
                print(f'Изменение границ {i+1}')
                while not self.find_opt(ratio=ratio, cond_r=cond_r)['success']:#В цикле сдвигаем границы
                    cond_r[i]=np.where(cond_r[i]>0, cond_r[i]*self.KOEF, cond_r[i]/self.KOEF)
                print(self.find_opt(ratio=ratio, cond_r=cond_r))
        #return self.find_opt(ratio=ratio, cond_r=cond_r)

In [4]:
datas=df.drop([0], axis=0).drop(['Unnamed: 0', 'Имя'], axis=1).astype(float).to_numpy()
datas

array([[1.000e+01, 9.000e-01, 1.111e+00, 1.500e-01, 5.000e-01, 2.000e-01,
        1.000e-01, 2.000e-03, 3.000e-03],
       [1.500e+01, 8.000e-01, 1.250e+00, 1.000e-01, 6.000e-01, 2.500e-01,
        1.500e-01, 3.000e-03, 4.000e-03],
       [5.000e+00, 4.000e-01, 2.500e+00, 5.000e-02, 3.000e-01, 4.000e-01,
        2.000e-01, 4.000e-03, 5.000e-03],
       [2.000e+01, 9.000e-01, 1.111e+00, 4.000e-01, 2.000e-01, 3.000e-01,
        2.500e-01, 5.000e-03, 6.000e-03]])

In [5]:
costs=datas[:, 0]
costs

array([10., 15.,  5., 20.])

In [6]:
voda=datas[:, 2:3]
voda

array([[1.111],
       [1.25 ],
       [2.5  ],
       [1.111]])

In [7]:
componets_for_div=datas[:, 2:]
componets_for_div

array([[1.111e+00, 1.500e-01, 5.000e-01, 2.000e-01, 1.000e-01, 2.000e-03,
        3.000e-03],
       [1.250e+00, 1.000e-01, 6.000e-01, 2.500e-01, 1.500e-01, 3.000e-03,
        4.000e-03],
       [2.500e+00, 5.000e-02, 3.000e-01, 4.000e-01, 2.000e-01, 4.000e-03,
        5.000e-03],
       [1.111e+00, 4.000e-01, 2.000e-01, 3.000e-01, 2.500e-01, 5.000e-03,
        6.000e-03]])

In [8]:
divided=componets_for_div/voda
divided

array([[1.        , 0.1350135 , 0.450045  , 0.180018  , 0.090009  ,
        0.00180018, 0.00270027],
       [1.        , 0.08      , 0.48      , 0.2       , 0.12      ,
        0.0024    , 0.0032    ],
       [1.        , 0.02      , 0.12      , 0.16      , 0.08      ,
        0.0016    , 0.002     ],
       [1.        , 0.360036  , 0.180018  , 0.270027  , 0.2250225 ,
        0.00450045, 0.00540054]])

In [9]:
divided=np.transpose(np.transpose(divided))
divided

array([[1.        , 0.1350135 , 0.450045  , 0.180018  , 0.090009  ,
        0.00180018, 0.00270027],
       [1.        , 0.08      , 0.48      , 0.2       , 0.12      ,
        0.0024    , 0.0032    ],
       [1.        , 0.02      , 0.12      , 0.16      , 0.08      ,
        0.0016    , 0.002     ],
       [1.        , 0.360036  , 0.180018  , 0.270027  , 0.2250225 ,
        0.00450045, 0.00540054]])

In [10]:
matrix=np.concatenate([datas[:,1].reshape((4, 1)), divided], axis=1)
matrix

array([[0.9       , 1.        , 0.1350135 , 0.450045  , 0.180018  ,
        0.090009  , 0.00180018, 0.00270027],
       [0.8       , 1.        , 0.08      , 0.48      , 0.2       ,
        0.12      , 0.0024    , 0.0032    ],
       [0.4       , 1.        , 0.02      , 0.12      , 0.16      ,
        0.08      , 0.0016    , 0.002     ],
       [0.9       , 1.        , 0.360036  , 0.180018  , 0.270027  ,
        0.2250225 , 0.00450045, 0.00540054]])

Первый индекс описывает продукт
1	Ячмень зерно	
2	Кукуруза зерно	
3	Силос	
4	Соя шрот
Второй индекс описывает компонент (кроме натурального, ушёл как коэф.)
1 Сухое
2 Натур
3 Сырой
4 Крахмал
5 NDF
6 ADF
7 Ca
8 Phos

## Пример1

In [11]:
products=[0, 1, 2]#Номера продуктов
bnd=[[0, 5], [0, 10], [0, float('inf')]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[20, None],[1.5, None], [6, None]]#Ограничения на компоненты
sootnosheniya=[[0.6, 1, 0, 0], [1, 0, 0.12, 0]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12

In [12]:
test1=method1(products,
             bnd,
             comps,
             comps_bnd,
             matrix,
             costs[products],
             sootnosheniya)

Если условие >=, то данные идут с плюсом.
Если <=, то надо умножить c -1.

In [13]:
opt=test1.find_opt()
opt

{'success': False,
 'x': None,
 'fun': None,
 'comps': None,
 'ratio': array([[0.6 , 1.  , 0.  , 0.  ],
        [1.  , 0.  , 0.12, 0.  ]]),
 'comps_bnd': array([-10. , -20. ,  -1.5,  -6. ])}

In [14]:
opt=test1.find_opt_change()
opt

{'success': True,
 'x': array([5.   , 8.549, 4.754]),
 'fun': 202.0,
 'comps': array([13.241, 18.303,  1.454,  6.924]),
 'ratio': array([[0.549, 1.   , 0.   , 0.   ],
        [1.   , 0.   , 0.11 , 0.   ]]),
 'comps_bnd': array([ -9.151, -18.303,  -1.373,  -5.491])}

In [15]:
test1.find_opt_change_one()

Изменение соотношения 2
{'success': True, 'x': array([5.   , 8.749, 6.251]), 'fun': 212.489, 'comps': array([14. , 20. ,  1.5,  7.2]), 'ratio': array([[0.6  , 1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.107, 0.   ]]), 'comps_bnd': array([-10. , -20. ,  -1.5,  -6. ])}


In [16]:
opt=test1.find_opt_change_ratio()
opt

{'success': True,
 'x': array([5.   , 8.749, 6.251]),
 'fun': 212.489,
 'comps': array([14. , 20. ,  1.5,  7.2]),
 'ratio': array([[0.533, 1.   , 0.   , 0.   ],
        [1.   , 0.   , 0.107, 0.   ]]),
 'comps_bnd': array([-10. , -20. ,  -1.5,  -6. ])}

In [17]:
opt=test1.find_opt_change_comp_bond()
opt

{'success': True,
 'x': array([5.   , 7.93 , 0.292]),
 'fun': 170.411,
 'comps': array([10.961, 13.222,  1.315,  6.092]),
 'ratio': array([[0.6 , 1.  , 0.  , 0.  ],
        [1.  , 0.  , 0.12, 0.  ]]),
 'comps_bnd': array([ -6.611, -13.222,  -0.992,  -3.967])}

## Пример 2

In [18]:
products=[0, 1, 2]#Номера продуктов
bnd=[[0, 5], [0, 10], [0, float('inf')]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[None, None],[2, None], [None, None]]#Ограничения на компоненты
sootnosheniya=[[0.6, 1, 0, 0]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12
test2=method1(products,
             bnd,
             comps,
             comps_bnd,
             matrix,
             costs[products],
             sootnosheniya)

In [19]:
opt=test2.find_opt()
opt

{'success': False,
 'x': None,
 'fun': None,
 'comps': None,
 'ratio': array([[0.6, 1. , 0. , 0. ]]),
 'comps_bnd': array([-10,  -2])}

In [20]:
opt=test2.find_opt_change()
opt

{'success': True,
 'x': array([ 5.   , 10.   , 20.506]),
 'fun': 302.531,
 'comps': array([20.702, 35.506,  1.885,  9.511]),
 'ratio': array([[0.566, 1.   , 0.   , 0.   ]]),
 'comps_bnd': array([-9.426, -1.885])}

In [21]:
opt=test2.find_opt_change_one()
opt

Изменение соотношения 1
{'success': True, 'x': array([ 5.   , 10.   , 26.247]), 'fun': 331.233, 'comps': array([22.999, 41.247,  2.   , 10.2  ]), 'ratio': array([[0.549, 1.   , 0.   , 0.   ]]), 'comps_bnd': array([-10,  -2])}
Изменение границ 2
{'success': True, 'x': array([5.   , 2.083, 9.583]), 'fun': 129.167, 'comps': array([10.   , 16.667,  1.033,  4.4  ]), 'ratio': array([[0.6, 1. , 0. , 0. ]]), 'comps_bnd': array([-10,  -1])}


## Пример 3

In [22]:
products=[0, 1, 2]#Номера продуктов
bnd=[[5, 5], [8.75, 8.75], [1.25, 1.25]]#Ограничения на продукты
comps=[0, 1, 2, 3]#Номера компонентов
comps_bnd=[[10, None],[15, None],[1.3, None], [None, None]]#Ограничения на компоненты
sootnosheniya=[[0.8, 1, 0, 0],  [0.2, 0, 0.0, 1]]#соотношения компонентов
#комп1/комп2>=0.6 комп3/комп1>=0.12

In [23]:
test3=method1(products,
             bnd,
             comps,
             comps_bnd,
             matrix,
             costs[products],
             sootnosheniya)
opt=test3.find_opt()
opt

{'success': True,
 'x': array([5.  , 8.75, 1.25]),
 'fun': 187.5,
 'comps': array([12. , 15. ,  1.4,  6.6]),
 'ratio': array([[0.8, 1. , 0. , 0. ],
        [0.2, 0. , 0. , 1. ]]),
 'comps_bnd': array([-10. , -15. ,  -1.3])}